<h1 > Offensive Language Classification </h1>


In [ ]:
#import all necessary library
import pandas as pd
import langdetect
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc
from collections import Counter
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tqdm import tqdm
from torch.optim import AdamW
from transformers import BertTokenizer, BertModel, get_scheduler
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve, auc



In [ ]:
#load all CSV file
train =pd.read_csv("../offence_data/train.csv")
test = pd.read_csv("../offence_data/test.csv")
valid = pd.read_csv("../offence_data/validation.csv") 
test_lebel = pd.read_csv("../offence_data/test_labels.csv") 


<h1 > just visualize train , test, validation data </h1>

In [ ]:
train.head()

In [ ]:
valid.head()

In [ ]:
test.head()

In [ ]:
test_lebel.head()

<h2 style="color: green;"> adding test label into test data and drop where lebel is NAN  </h2>

In [ ]:
test['Index'] = range(1, len(test) + 1)
test_lebel['Index'] = range(1, len(test_lebel) + 1)
test = test.merge(test_lebel[['Index', 'toxic']], on='Index', how='left')
test.drop('Index', axis=1, inplace=True)
test['toxic'] = test['toxic'].dropna()
test = test.dropna(subset=['toxic'])
test['toxic'] = test['toxic'].astype(int)
test = test.rename(columns={'content': 'feedback_text'})
print(test.tail(5))

<h1 style="color: green;"> Start EDA and Data manipulation   </h1>

In [ ]:
train.head()

In [ ]:
#Lebel destribuition chart
label_cols = ['toxic', 'abusive', 'vulgar', 'menace', 'offense', 'bigotry']
train[label_cols].sum().plot(kind="bar", title="Label Distribution")
plt.show()



<h1> Detect laguages present in train dataset </h1>

In [ ]:
from langdetect import detect, LangDetectException
import pandas as pd

# Sample DataFrame (replace this with your real one)
# train = pd.read_csv('your_file.csv')

# Create a list to hold detected languages
detected_languages = []

# Iterate through each feedback text
for text in train['feedback_text']:
    if pd.isna(text):
        detected_languages.append(None)
    else:
        try:
            lang = detect(text)
            detected_languages.append(lang)
        except LangDetectException:
            detected_languages.append(None)  # Or use a default like 'unknown'

# Add the language column to the DataFrame
train['lang'] = detected_languages


In [ ]:
print(train['lang'].isnull().sum())

In [ ]:
train['lang'].unique()


In [ ]:
train = train.dropna(subset=['lang'])

In [ ]:
all_words = ' '.join(train['lang'].astype(str)).lower().split()


common_words = Counter(all_words)
common_words_df = pd.DataFrame(common_words.most_common(20), columns=['word', 'count'])

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(data=common_words_df, x='count', y='word', palette='mako')
plt.title("Top 20 Most Common Language in training set")
plt.xlabel("Count")
plt.ylabel("Word")
plt.show()

<h1>collecting multi langual stop word for further analisys </h1>

In [ ]:

nltk.download('stopwords')

# List of language names (without language codes)
languages = [
    'english', 'spanish', 'tagalog', 'estonian', 'german', 'persian (farsi)', 'afrikaans',
    'norwegian', 'somali', 'indonesian', 'french', 'vietnamese', 'slovenian', 'turkish',
    'portuguese', 'romanian', 'swahili', 'croatian', 'danish', 'albanian', 'welsh', 'italian',
    'czech', 'swedish', 'finnish', 'dutch', 'arabic', 'polish', 'bengali', 'catalan', 'hungarian',
    'lithuanian', 'slovak', 'russian', 'hebrew', 'korean', 'chinese (simplified)', 'gujarati', 
    'tamil', 'greek', 'hindi', 'thai', 'latvian', 'macedonian', 'malayalam', 'chinese (traditional)',
    'telugu', 'marathi', 'bulgarian'
]

multi_stop_words = set()

# Loop through the list of languages and update the multi_stop_words set
for lang in languages:
    try:
        # Check if stopwords exist for the language
        if lang in stopwords.fileids():
            multi_stop_words.update(stopwords.words(lang))
        else:
            print(f"No stopwords available for: {lang}")
    except Exception as e:
        print(f"Error with language {lang}: {e}")

print(f"Collected {len(multi_stop_words)} stopwords.")



In [ ]:

# Check for Missing Values
print("Missing values:\n", train.isnull().sum())

# Plot bar chart for missing values
missing_counts = train.isnull().sum()
missing_counts = missing_counts[missing_counts >= 0]  

if not missing_counts.empty:
    plt.figure(figsize=(10, 5))
    sns.barplot(x=missing_counts.index, y=missing_counts.values, palette="rocket")
    plt.title("Missing Values per Column")
    plt.ylabel("Number of Missing Values")
    plt.xlabel("Column Name")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Drop rows with missing feedback_text if any
missing_texts = train[train['feedback_text'].isnull()]
print(f"\nRows with missing feedback_text: {len(missing_texts)}")
if len(missing_texts) > 0:
    train = train.dropna(subset=['feedback_text'])

# Word Distribution and Sentence Length

train['word_count'] = train['feedback_text'].apply(lambda x: len(str(x).split()))
train['char_count'] = train['feedback_text'].apply(lambda x: len(str(x)))

# Plot histograms
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.histplot(train['word_count'], bins=50, ax=axes[0], color='skyblue')
axes[0].set_title('Word Count Distribution')

sns.histplot(train['char_count'], bins=50, ax=axes[1], color='salmon')
axes[1].set_title('Character Count Distribution')
plt.tight_layout()
plt.show()

# Most Common Words (Before Preprocessing)


# Get the list of English stopwords
# Remove stop words from feedback_text
all_words = ' '.join(train['feedback_text'].astype(str)).lower().split()
filtered_words = [word for word in all_words if word not in multi_stop_words]

# Count the most common words
common_words = Counter(filtered_words)
common_words_df = pd.DataFrame(common_words.most_common(20), columns=['word', 'count'])

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(data=common_words_df, x='count', y='word', palette='mako')
plt.title("Top 20 Most Common Words (after removing multi langual  Stopwords only)")
plt.xlabel("Count")
plt.ylabel("Word")
plt.show()


In [ ]:


# Function to clean the text (remove stopwords, punctuation, numbers, etc.)
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation using str.translate
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers and special characters using regular expressions
    text = re.sub(r'\d+', '', text)

    # Remove stopwords
    words = text.split()
    cleaned_words = [word for word in words if word not in multi_stop_words ]

    return ' '.join(cleaned_words)

# Apply the clean_text function to the feedback_text column
train['cleaned_feedback_text'] = train['feedback_text'].apply(lambda x: clean_text(str(x)))

# Combine all cleaned words
all_words = ' '.join(train['cleaned_feedback_text']).split()

# Count the most common words
common_words = Counter(all_words)
common_words_df = pd.DataFrame(common_words.most_common(20), columns=['word', 'count'])

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(data=common_words_df, x='count', y='word', palette='mako')
plt.title("Top 20 Most Common Words (After removing stopword and other text noise)")
plt.xlabel("Count")
plt.ylabel("Word")
plt.show()


<h1 style="color: green;"> Word Frequency Map  </h1>

In [ ]:
from wordcloud import WordCloud

# Combine all feedback_text into one string
all_text = ' '.join(train['feedback_text'].astype(str)).lower()

# Generate Word Cloud
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    stopwords=multi_stop_words,
    max_words=200,
    colormap='plasma'
).generate(all_text)

# Plot Word Cloud
plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("🔤 Word Frequency Map (Word Cloud)", fontsize=20)
plt.show()


<h1> balancing the  dataset </h1>

In [ ]:
#badly needed but donot get time to do it 

<h1>preprocessing , training , evaluation using multilangual pretrained model  bert-base-multilingual-cased <h1>

In [ ]:

warnings.filterwarnings("ignore")

# NLTK setup
nltk.download('stopwords')
nltk.download('wordnet')


# Preprocessing

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'@\w+|\#', '', text)
    text = re.sub(rf"[{re.escape(string.punctuation)}]", "", text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)


# Load Data

train_df = train
val_df = valid
test_df = test

# Apply preprocessing
train_df['feedback_text'] = train_df['feedback_text'].apply(preprocess)
val_df['feedback_text'] = val_df['feedback_text'].apply(preprocess)
test_df['feedback_text'] = test_df['feedback_text'].apply(preprocess)

# Label columns
label_cols = ['toxic', 'abusive', 'vulgar', 'menace', 'offense', 'bigotry']


# Tokenization and Dataset

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

class ToxicDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

train_dataset = ToxicDataset(train_df['feedback_text'].tolist(), train_df[label_cols].values.tolist())
val_dataset = ToxicDataset(val_df['feedback_text'].tolist(), val_df[['toxic']].values.tolist())
test_dataset = ToxicDataset(test_df['feedback_text'].tolist(), test_df[['toxic']].values.tolist())


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)


# Model

class ToxicClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, 6)  # 6 labels

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = self.dropout(output.pooler_output)
        return torch.sigmoid(self.fc(pooled_output))


# Training Setup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ToxicClassifier().to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 1
loss_fn = nn.BCELoss()
total_steps = len(train_loader) * epochs
scheduler = get_scheduler("linear", optimizer, num_warmup_steps=0, num_training_steps=total_steps)


# Training Loop

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {total_loss / len(train_loader):.4f}")

# Validation Evaluation (TOXIC only)
model.eval()
val_preds, val_trues = [], []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].cpu().numpy()
        outputs = model(input_ids, attention_mask).cpu().numpy()
        val_preds.extend(outputs[:, 0])  # Only TOXIC
        val_trues.extend(labels[:, 0])  # Only TOXIC

val_preds = np.array(val_preds)
val_trues = np.array(val_trues)
val_bin_preds = (val_preds >= 0.5).astype(int)

print("\n Validation Classification Report (TOXIC only):")
print(classification_report(val_trues, val_bin_preds))
print("AUC:", roc_auc_score(val_trues, val_preds))

# ROC curve for TOXIC
fpr, tpr, _ = roc_curve(val_trues, val_preds)
plt.figure()
plt.plot(fpr, tpr, label="TOXIC AUC = %0.4f" % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (TOXIC - Validation)')
plt.legend()
plt.grid()
plt.show()

# Confusion Matrix for Test Set
sns.heatmap(confusion_matrix(val_trues, val_bin_preds), annot=True, fmt="d", cmap="Greens")
plt.title("Confusion Matrix - valid Set (Toxic)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
# Test Prediction (TOXIC Only)
t_preds, t_trues = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].cpu().numpy()
        outputs = model(input_ids, attention_mask).cpu().numpy()
        t_preds.extend(outputs[:, 0])  # Only TOXIC
        t_trues.extend(labels[:, 0])  # Only TOXIC

t_preds = np.array(t_preds)
t_trues = np.array(t_trues)
t_bin_preds = (t_preds >= 0.5).astype(int)

print("\n Validation Classification Report (TOXIC only):")
print(classification_report(t_trues, t_bin_preds))
print("AUC:", roc_auc_score(t_trues, t_preds))

# ROC curve for TOXIC
fpr, tpr, _ = roc_curve(t_trues, t_preds)
plt.figure()
plt.plot(fpr, tpr, label="TOXIC AUC = %0.4f" % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (TOXIC - Test)')
plt.legend()
plt.grid()
plt.show()

# Confusion Matrix for Test Set
sns.heatmap(confusion_matrix(t_trues, t_bin_preds), annot=True, fmt="d", cmap="Greens")
plt.title("Confusion Matrix - Test Set (Toxic)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()